# 📊 Phase 2 — EDA: Klarna Stock & IPO Analysis
### BNPL Under the Microscope

**Objective:** Deep exploratory analysis of Klarna's stock performance since IPO.

**Key Questions We Answer:**
1. How did KLAR perform vs peers and the S&P 500 since IPO?
2. What does the valuation journey ($45B → $6.7B → $15B → $7.8B) look like statistically?
3. What are the return/risk characteristics of KLAR post-IPO?
4. Where are the key support/resistance levels and what events caused breaks?
5. Is Klarna fundamentally cheap or fairly priced vs peers today?

**Key Insight We're Building Toward:**
> Klarna beat revenue estimates in Q3 2025 (+26% YoY) but still dropped 9% on earnings day. This disconnect — strong fundamentals, weak market reception — is the central tension of this project.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Style Configuration ───────────────────────────────────────────────────────
plt.style.use('dark_background')

COLORS = {
    'gold':   '#E8C56D',
    'red':    '#FF5B5B',
    'green':  '#6FCF97',
    'blue':   '#4A9EFF',
    'orange': '#C45C3A',
    'muted':  '#7A7A8A',
    'bg':     '#0A0A0C',
    'surface':'#111116',
}

TICKER_COLORS = {
    'KLAR': COLORS['gold'],
    'AFRM': COLORS['green'],
    'PYPL': COLORS['blue'],
    'SQ':   COLORS['orange'],
    '^GSPC': COLORS['muted'],
}

plt.rcParams.update({
    'figure.facecolor':  COLORS['bg'],
    'axes.facecolor':    COLORS['surface'],
    'axes.edgecolor':    '#2A2A35',
    'axes.labelcolor':   '#7A7A8A',
    'xtick.color':       '#7A7A8A',
    'ytick.color':       '#7A7A8A',
    'grid.color':        '#1E1E28',
    'grid.linestyle':    '--',
    'grid.linewidth':    0.5,
    'font.family':       'monospace',
    'font.size':         10,
    'figure.dpi':        150,
})

RAW_DIR = '../data/raw'
PROC_DIR = '../data/processed'
VIS_DIR  = '../visuals'
import os
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

print('✅ Config ready.')

In [ ]:
# ── Load Data ─────────────────────────────────────────────────────────────────
prices      = pd.read_csv(f'{RAW_DIR}/stock_prices_raw.csv', index_col=0, parse_dates=True)
prices_norm = pd.read_csv(f'{RAW_DIR}/stock_prices_normalised.csv', index_col=0, parse_dates=True)
kl_annual   = pd.read_csv(f'{RAW_DIR}/klarna_annual_fundamentals.csv')
kl_qtr      = pd.read_csv(f'{RAW_DIR}/klarna_quarterly.csv')
val_hist    = pd.read_csv(f'{RAW_DIR}/klarna_valuation_history.csv')

print(f'Stock prices: {prices.shape}')
print(f'Klarna quarterly: {kl_qtr.shape}')
print(prices.tail(3))

## 2.1 — Relative Performance Since IPO

Compare KLAR vs AFRM, PYPL, SQ, and S&P 500, all indexed to 100 on IPO day.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

labels = {'KLAR': 'Klarna (KLAR)', 'AFRM': 'Affirm (AFRM)',
          'PYPL': 'PayPal (PYPL)', 'SQ': 'Block (SQ)', '^GSPC': 'S&P 500'}

for ticker in ['KLAR', 'AFRM', 'PYPL', 'SQ', '^GSPC']:
    lw = 2.5 if ticker == 'KLAR' else 1.5
    alpha = 1.0 if ticker == 'KLAR' else 0.7
    ls = '-' if ticker in ['KLAR', '^GSPC'] else '--'
    ax.plot(prices_norm.index, prices_norm[ticker],
            color=TICKER_COLORS[ticker], linewidth=lw,
            alpha=alpha, linestyle=ls, label=labels[ticker])

# Key event annotations
events = [
    ('2025-11-18', 'Q3 Earnings\n−9% day', -8),
    ('2025-12-04', 'KLAR peaks\n$57.20', +5),
    ('2026-01-28', 'Litigation\nnews', -8),
]

for date_str, label, yoffset in events:
    date = pd.Timestamp(date_str)
    if date in prices_norm.index:
        y = prices_norm.loc[date, 'KLAR']
        ax.annotate(label,
            xy=(date, y), xytext=(date, y + yoffset),
            color=COLORS['muted'], fontsize=8,
            arrowprops=dict(arrowstyle='->', color=COLORS['muted'], lw=0.8),
            ha='center')

# Reference line at 100
ax.axhline(100, color='#2A2A35', linewidth=1, linestyle=':')
ax.text(prices_norm.index[1], 101, 'IPO Price Baseline', color='#2A2A35', fontsize=8)

ax.set_title('Relative Performance Since Klarna IPO (Sep 10, 2025 = 100)',
             color='white', fontsize=14, pad=16, loc='left')
ax.set_ylabel('Indexed Price (100 = IPO Day)', labelpad=12)
ax.set_xlabel('')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.grid(True, alpha=0.5)
ax.legend(loc='upper right', framealpha=0.2, fontsize=9)

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/01_relative_performance.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n💾 Saved: visuals/01_relative_performance.png')

## 2.2 — Klarna Returns Distribution & Volatility

In [ ]:
# ── Compute daily returns ─────────────────────────────────────────────────────
returns = prices.pct_change().dropna()

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# — Plot 1: KLAR daily returns distribution
ax = axes[0]
klar_ret = returns['KLAR'].dropna()
ax.hist(klar_ret * 100, bins=35, color=COLORS['gold'],
        alpha=0.7, edgecolor='none')
ax.axvline(klar_ret.mean() * 100, color=COLORS['red'], linestyle='--', linewidth=1.5, label=f'Mean: {klar_ret.mean()*100:.2f}%')
ax.axvline(0, color=COLORS['muted'], linestyle=':', linewidth=1)
ax.set_title('KLAR Daily Returns Distribution', color='white', fontsize=11)
ax.set_xlabel('Daily Return (%)')
ax.legend(fontsize=8)

# — Plot 2: Rolling 14-day volatility comparison
ax = axes[1]
roll_vol = returns[['KLAR', 'AFRM', '^GSPC']].rolling(14).std() * np.sqrt(252) * 100
for col in roll_vol.columns:
    color = TICKER_COLORS[col]
    label = {'KLAR': 'Klarna', 'AFRM': 'Affirm', '^GSPC': 'S&P 500'}[col]
    ax.plot(roll_vol.index, roll_vol[col], color=color, linewidth=1.5, label=label)
ax.set_title('Rolling 14-Day Annualised Volatility', color='white', fontsize=11)
ax.set_ylabel('Annualised Vol (%)')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax.legend(fontsize=8)
ax.grid(True, alpha=0.4)

# — Plot 3: Summary risk/return scatter
ax = axes[2]
summary = {}
for t in ['KLAR', 'AFRM', 'PYPL', 'SQ', '^GSPC']:
    r = returns[t].dropna()
    summary[t] = {
        'ann_return': r.mean() * 252 * 100,
        'ann_vol': r.std() * np.sqrt(252) * 100,
    }
s = pd.DataFrame(summary).T

for t, row in s.iterrows():
    color = TICKER_COLORS[t]
    ax.scatter(row['ann_vol'], row['ann_return'], color=color, s=120, zorder=5)
    ax.annotate(t, (row['ann_vol'], row['ann_return']),
                textcoords='offset points', xytext=(8, 4),
                color=color, fontsize=9)

ax.axhline(0, color='#2A2A35', linestyle='--', linewidth=0.8)
ax.set_title('Risk vs Return (Since IPO, Annualised)', color='white', fontsize=11)
ax.set_xlabel('Annualised Volatility (%)')
ax.set_ylabel('Annualised Return (%)')
ax.grid(True, alpha=0.4)

plt.suptitle('Klarna Post-IPO Risk & Return Analysis', color='white',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(f'{VIS_DIR}/02_risk_return_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n📊 Risk/Return Summary:')
print(s.round(2))

## 2.3 — Valuation Lifecycle Chart

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

val = val_hist.copy()
x = range(len(val))
colors = []
for _, row in val.iterrows():
    if row['valuation_b'] == val['valuation_b'].max():
        colors.append(COLORS['gold'])
    elif row['valuation_b'] == val['valuation_b'].min():
        colors.append(COLORS['red'])
    elif row['type'] == 'public':
        colors.append(COLORS['green'])
    else:
        colors.append(COLORS['muted'])

bars = ax.bar(x, val['valuation_b'], color=colors, width=0.65,
              edgecolor='none', zorder=3)

# Value labels
for i, (idx, row) in enumerate(val.iterrows()):
    ax.text(i, row['valuation_b'] + 0.8, f"${row['valuation_b']}B",
            ha='center', va='bottom', color='white', fontsize=9, fontweight='bold')

# Price labels where available
for i, (idx, row) in enumerate(val.iterrows()):
    if pd.notna(row.get('share_price_equiv')):
        ax.text(i, -3.5, f"${row['share_price_equiv']}/sh",
                ha='center', va='top', color=COLORS['muted'], fontsize=8)

ax.set_xticks(list(x))
ax.set_xticklabels(
    [f"{r['event']}\n{r['date'][:7]}" for _, r in val.iterrows()],
    fontsize=8, rotation=10
)
ax.set_ylabel('Valuation (USD Billions)')
ax.set_title('Klarna Valuation Lifecycle — From $5.5B to $45.6B to $7.8B',
             color='white', fontsize=14, loc='left', pad=16)

# Legend
handles = [
    mpatches.Patch(color=COLORS['gold'], label='Peak ($45.6B)'),
    mpatches.Patch(color=COLORS['red'], label='Trough ($6.7B)'),
    mpatches.Patch(color=COLORS['green'], label='Public market'),
    mpatches.Patch(color=COLORS['muted'], label='Private round'),
]
ax.legend(handles=handles, loc='upper left', framealpha=0.2, fontsize=9)
ax.grid(True, axis='y', alpha=0.4)
ax.set_ylim(-6, 52)

plt.tight_layout()
plt.savefig(f'{VIS_DIR}/03_valuation_lifecycle.png', dpi=150, bbox_inches='tight')
plt.show()

## 2.4 — Revenue & Profitability Deep Dive

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# — Plot 1: Annual Revenue with growth overlay
ax1 = axes[0, 0]
ax1b = ax1.twinx()
bar_colors = [COLORS['muted']] * 5 + [COLORS['gold'], COLORS['green']]
ax1.bar(kl_annual['year'], kl_annual['revenue_m'], color=bar_colors, width=0.6, zorder=3)
valid_growth = kl_annual.dropna(subset=['revenue_growth_pct'])
ax1b.plot(valid_growth['year'], valid_growth['revenue_growth_pct'],
          color=COLORS['gold'], marker='o', linewidth=2, markersize=5,
          linestyle='--', alpha=0.8)
ax1.set_title('Annual Revenue ($M) & YoY Growth %', color='white', fontsize=11)
ax1.set_ylabel('Revenue ($M)')
ax1b.set_ylabel('Growth %', color=COLORS['gold'])
ax1b.tick_params(colors=COLORS['gold'])
ax1.grid(True, axis='y', alpha=0.3)

# — Plot 2: GMV vs Net Income (quarterly)
ax2 = axes[0, 1]
ax2b = ax2.twinx()
ax2.bar(range(len(kl_qtr)), kl_qtr['gmv_b'],
        color=COLORS['blue'], alpha=0.6, width=0.6, label='GMV ($B)')
ni_colors = [COLORS['green'] if v >= 0 else COLORS['red'] for v in kl_qtr['net_income_m']]
ax2b.bar([i + 0.0 for i in range(len(kl_qtr))], kl_qtr['net_income_m'],
         color=ni_colors, alpha=0.8, width=0.3, label='Net Income ($M)')
ax2.set_xticks(range(len(kl_qtr)))
ax2.set_xticklabels(kl_qtr['quarter'], rotation=45, ha='right', fontsize=8)
ax2.set_title('GMV ($B) vs Net Income ($M) — Quarterly', color='white', fontsize=11)
ax2.set_ylabel('GMV ($ Billions)', color=COLORS['blue'])
ax2b.set_ylabel('Net Income ($M)', color=COLORS['red'])
ax2.tick_params(axis='y', colors=COLORS['blue'])
ax2b.tick_params(axis='y', colors=COLORS['red'])
ax2b.axhline(0, color='#2A2A35', linewidth=1)
ax2.grid(True, axis='y', alpha=0.3)

# — Plot 3: ARPU trend (revenue per user)
ax3 = axes[1, 0]
ax3.fill_between(kl_annual['year'], kl_annual['arpu'],
                  alpha=0.15, color=COLORS['gold'])
ax3.plot(kl_annual['year'], kl_annual['arpu'],
         color=COLORS['gold'], linewidth=2.5, marker='o', markersize=6)
for _, row in kl_annual.iterrows():
    ax3.annotate(f"${row['arpu']:.1f}",
                 (row['year'], row['arpu']),
                 textcoords='offset points', xytext=(0, 10),
                 ha='center', color=COLORS['muted'], fontsize=8)
ax3.set_title('Average Revenue Per User ($) — ARPU Trend', color='white', fontsize=11)
ax3.set_ylabel('ARPU ($)')
ax3.set_xlabel('Year')
ax3.grid(True, alpha=0.3)

# — Plot 4: Take Rate (Revenue / GMV)
ax4 = axes[1, 1]
ax4.fill_between(kl_annual['year'], kl_annual['take_rate_pct'],
                  alpha=0.15, color=COLORS['green'])
ax4.plot(kl_annual['year'], kl_annual['take_rate_pct'],
         color=COLORS['green'], linewidth=2.5, marker='s', markersize=6)
for _, row in kl_annual.iterrows():
    ax4.annotate(f"{row['take_rate_pct']:.2f}%",
                 (row['year'], row['take_rate_pct']),
                 textcoords='offset points', xytext=(0, 10),
                 ha='center', color=COLORS['muted'], fontsize=8)
ax4.set_title('Take Rate % (Revenue / GMV) — Monetisation Efficiency', color='white', fontsize=11)
ax4.set_ylabel('Take Rate (%)')
ax4.set_xlabel('Year')
ax4.grid(True, alpha=0.3)

plt.suptitle('Klarna Fundamentals Deep Dive', color='white', fontsize=16, y=1.01)
plt.tight_layout()
plt.savefig(f'{VIS_DIR}/04_fundamentals_deepdive.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n💾 Saved: visuals/04_fundamentals_deepdive.png')

## 2.5 — Key Statistical Summary

In [ ]:
# ── Compute summary stats for report ─────────────────────────────────────────
klar = prices['KLAR'].dropna()
ipo_price = 40.0
peak_price = 57.20
current = klar.iloc[-1]

stats = {
    'IPO Price':                f'${ipo_price:.2f}',
    'Day 1 Close':              f'${klar.iloc[0]:.2f}  ({(klar.iloc[0]/ipo_price-1)*100:+.1f}% vs IPO)',
    'All-time High (post-IPO)': f'${peak_price:.2f}',
    'Current Price':            f'${current:.2f}',
    'Return from IPO price':    f'{(current/ipo_price-1)*100:.1f}%',
    'Return from ATH':          f'{(current/peak_price-1)*100:.1f}%',
    'Days since IPO':           str(len(klar)),
    'Best single day':          f'{returns["KLAR"].max()*100:+.1f}%',
    'Worst single day':         f'{returns["KLAR"].min()*100:+.1f}%',
    'Avg daily return':         f'{returns["KLAR"].mean()*100:.3f}%',
    'Daily volatility':         f'{returns["KLAR"].std()*100:.2f}%',
    'Annualised volatility':    f'{returns["KLAR"].std()*np.sqrt(252)*100:.1f}%',
    'Sharpe Ratio (0% RF)':     f'{(returns["KLAR"].mean()/returns["KLAR"].std()*np.sqrt(252)):.3f}',
    'Max Drawdown':             f'{((klar / klar.cummax()) - 1).min()*100:.1f}%',
}

print('\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print('  KLARNA (KLAR) — POST-IPO STATISTICAL SUMMARY')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
for k, v in stats.items():
    print(f'  {k:<35}  {v}')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')

# Save to processed
pd.DataFrame(stats.items(), columns=['metric', 'value']).to_csv(
    f'{PROC_DIR}/klar_stats_summary.csv', index=False)
print('\n✅ Phase 2 complete. Proceed to 03_bnpl_debt_risk.ipynb')